## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
from Requirements import *

In [2]:
convention_db = sqlite3.connect("Data/2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [36]:
convention_data = []
# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 


query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions;
                            ''')

#print('\nData in Conventions Table:')
for row in query_results :
    convention_data.append([" ".join(prepare(row[0], pipeline = my_pipeline)), row[1]])


Let's look at some random entries and see if they look right. 

In [7]:
random.choices(convention_data,k=10)

[['i’m congresswoman lisa blunt rochester history class future children learning moment they’re learning pain grief worry they’re also learning man named joe biden restored decency government integrity democracy',
  'Democratic'],
 ['that’s good every american everyone board', 'Democratic'],
 ['virginia', 'Republican'],
 ['well called told guy friend town went joe', 'Democratic'],
 ['he’s got courage he’s got tremendous courage character judgment tough witnessed close',
  'Democratic'],
 ['mr chairman jaime harrison candidate united states senate speaking campus south carolina state university alma mater majority whip jim clyburn late dr emily england clyburn honors college named proud hbcu contributed 22 general officers armed services tonight proudly cast south carolina’s 64 votes follows 15 bernie sanders 49 next president united states joe biden',
  'Democratic'],
 ['they’re long laundry list uncertainties juggle', 'Democratic'],
 ['thank pursuant convention rules we’ll proceed rol

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [8]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [18]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    ret_dict = dict()
    for token in text.split():
        if token in fw:
            ret_dict[token] = True
    return(ret_dict)

In [11]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [12]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [13]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [14]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [15]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Without trying to sound too political, a lot of the most informative features listed with the likelihood that the text stated is republican or democrat, makes sense. Broadcasted around media, the words used goes along with specific parties. I found this interesting to see these features lined up to the party ratio.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [19]:
cong_db = sqlite3.connect("Data/congressional_data.db")
cong_cur = cong_db.cursor()

In [20]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [37]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

print('\nCandidate Data in Congressional Tweets:')
for row in results :
    tweet_data.append([" ".join(prepare(row[2], pipeline = my_pipeline)), row[1]])



Candidate Data in Congressional Tweets:


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [27]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [31]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: bearlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bgo tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: bapparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bwexe2x80x99re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives linennhttpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: bletxe2x80x99s make even greater kag xf0x9fx87xbaxf0x9fx87xb8 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Re

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [32]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [33]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3605, 'Democratic': 673}),
             'Democratic': defaultdict(int,
                         {'Republican': 4723, 'Democratic': 1001})})

### Reflections

For the results, it seems that the Naive-Bayes classifier model is more accurate for Republicans in classified correctly. For the Democratic dictionary, it's inverted where most are classified as Republican instead of Democratic. It makes me think that it might be because there isn't enough data for it to be properly trained so the precision isn't there yet, or too many features causing overfitting.